In [19]:
import pandas as pd
import numpy as np

In [20]:
data = pd.read_csv("austin_final.csv")
# the features or the 'x' values of the data
# these columns are used to train the model
# the last column, i.e, precipitation column 
# will serve as the label 
X = data.drop(['PrecipitationSumInches'], axis = 1)

In [21]:
Y = data['PrecipitationSumInches']

# reshaping it into a 2-D vector

Y = Y.values.reshape(-1, 1)

In [22]:
X = X.filter(['TempAvgF', 'DewPointAvgF', 'HumidityAvgPercent',
                  'SeaLevelPressureAvgInches', 'VisibilityAvgMiles',
                  'WindAvgMPH'], axis = 1)

In [24]:
# def train_validate_test_split_v2(X,Y):
#     train_percent=.7
#     validate_percent=.1
#     np.random.seed(seed=None)
#     perm_x = np.random.permutation(X.index)
#     prem_y = np.random.permutation(Y.index)
#     m_x = len(X.index)
#     m_y = len(Y.index)

#     train_end_x = int(train_percent * m_x)
#     train_end_y = int(train_percent * m_y)
#     validate_end_x = int(validate_percent * m_x) + train_end_x
#     validate_end_y = int(validate_percent * m_y) + train_end_y

#     train_x = X.iloc[perm_x[:train_end_x]]
#     validate_x = X.iloc[perm_x[train_end_x:validate_end_x]]
#     test_x = X.iloc[perm_x[validate_end_x:]]
#     train_y = Y.iloc[prem_y[:train_end_y]]
#     validate_y = Y.iloc[prem_y[train_end_y:validate_end_y]]
#     test_y = Y.iloc[prem_y[validate_end_y:]]
    
#     return train_x, train_y, validate_x, validate_y, test_x, test_y

In [25]:
def train_val_test_split(X, Y):
    p = np.random.permutation(len(Y))
    tr = np.floor(len(Y)*0.7).astype('int')
    te = np.floor(len(Y)*0.8).astype('int')
    X_train = X[p[:tr],:]
    Y_train = Y[p[:tr]]
    X_val = X[p[tr+1:te],:]
    Y_val = Y[p[tr+1:te]]
    X_test = X[p[te+1:],:]
    Y_test = Y[p[te+1:]]
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [29]:
# Convert x to numpy array
X = np.array(X)
Y = np.array(Y)

In [31]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = train_val_test_split(X, Y)

In [32]:
from sklearn.linear_model import LinearRegression

In [33]:
LR = LinearRegression()

In [34]:
LR.fit(X_train, Y_train)

LinearRegression()

In [36]:
print(LR.intercept_, LR.coef_)

[0.38725348] [[-0.00589098  0.00706588  0.00347827  0.00678657 -0.0831539   0.01368794]]


In [37]:
Yhat_skl_val = LR.predict(X_val)

In [38]:
theta = np.matmul(np.matmul(np.linalg.inv(np.matmul(X_train.transpose(), X_train)), X_train.transpose()), Y_train)
#Analytical Solution =  (XX^T)^-1 * X^T * Y

In [39]:
theta

array([[-0.00354778],
       [ 0.00472326],
       [ 0.0048985 ],
       [ 0.01469104],
       [-0.08086706],
       [ 0.01393733]])

In [ ]:
# To get the intercept, we need to add the augmented column of 1s to the X_train